In [1]:
import pandas as pd
import os
array= []
r = pd.read_csv("Desktop/PhD project/CISBP_results/entiredata_2020_04_14_1-09_pm/Zscores.txt", sep = "\t")


'''
Outputs a table containing top 5 entries
Outputs column names and no. of values they hold
Outputs column names and no. of values they hold
'''


r2 = pd.DataFrame(r)

'''
Converting the content into a DataFrame explicitly.

For sequence scans, we performed a one-sided Z test for each motif on its sequence scores, and defined ‘strong motif matches’ as those with scores significantly higher than the mean (FDR <0.1, corrected for all motifs).
Ref : "A compendium of RNA-binding motifs for decoding gene regulation Debashish Ray, Hilal Kazan, […]Timothy R. Hughes"
'''
feat = r2.describe()
Upper_quart_seqscan = feat.loc['75%':] #Deciding cutoff for each motif. Values in the upper quartile and beyond. 

'''
Any value above the cutoff for a particular motif would make the corresponding 7mer a strong match. 
Whereas for every 7mer, the highest value across the row would mean the strongest match is with that particular motif amongst all other motifs(i.e. the other ones for which this 7mer passed the cutoff)

'''
Cutoffs = Upper_quart_seqscan.loc['75%'] # The final cutoff list (containing cutoffs for each motif column).



In [10]:
'''
Additional files which can be found in CISBP downloads
'''
rbp_info = pd.read_csv("Desktop/PhD project/CISBP_results/entiredata_2020_04_14_1-09_pm/RBP_Information_all_motifs.txt", sep="\t")
rbp_info

In [3]:
rbp_info2 = pd.read_csv("Desktop/PhD project/CISBP_Results/entiredata_2020_04_14_1-09_pm/RBP_Information.txt", sep = "\t")
rbp_info2

In [5]:
'''
Experimenting with stuff. (Not important)

import numpy as np
r2['7mer'] #Accessing columns
w = r2[r2['7mer']=='AAAAAAG'].index.values

int.from_bytes((w.tostring()), byteorder='big')
'''


"\nExperimenting with stuff. (Not important)\n\nimport numpy as np\nr2['7mer'] #Accessing columns\nw = r2[r2['7mer']=='AAAAAAG'].index.values\n\nint.from_bytes((w.tostring()), byteorder='big')\n"

In [11]:
'''
Searching repressor barcode and cis-elements within the Z-score DataFrame. Barcode and cis-element IDs are appeneded with the respective elements.

'''
Repressors = pd.Series(['CGAGGAGTCC_9e','ATCATGGCGC_11e','CAGATAACGA_15e','ACCATCCTGA_20e','CATAGATCGC_26e','CATTCAGCCC_28e','CACAGAAGGC_33e','CAAAATAAAC_37e','AACATGACGC_38e','AATCACCAAC_39e','CAATTGCGAT_40e','CAAATGAGAA_52e','CAGATAAACA_54e','AGACTAATTA_56e','CGGGGCTCAA_60e','AAAAATGCGA_61e','ACCTCCTCGC_62e','CGCACAGATG_68e','GCGCTACCCG_103e','GCAACACCTC_104e','GCGGTCACGT_110e','GCCAGTTTCT_9B','TGCCACTCAT_11B','TAAAACACTT_15B','CGATGAGAAG_20B','TAATTCAGTA_26B','ATCAGACCCA_28B','GAGTGAGGAA_33B','ACGCGCGGGC_37B','CATGGTAGGC_38B','TAATGGCGTA_39B','TTTACCTAGC_40B','AACAAGCAGC_52B','TTGACGGCGC_54B','AAGGGCCTCG_56B','AGAAGGATAT_60B','GAGAAGGCAG_61B','GGAAAGTCAC_62B','GGGTTCGAGA_68B','ATGATGTACG_103B','TGTCGCATCC_104B','ACAACAAAGG_110B'])


Cons = {}
Cons2 = {}
Segul = []
for a in Repressors:
    colx = []
    Rep_motifs = []
    All_R_Motifs = []
    All_Motifs = []
    Other_matches = []
    for c in r2['7mer']:
        if c in a:
            z = (r2.loc[r2[r2['7mer']==c].index.values]).T
            l=1
            maxval = 0
            while (l < 245):
                if maxval < z.iloc[l].values:# Looking for the strongest motif match among all other motifs
                    maxval = z.iloc[l].values
                    Sev_mer= pd.DataFrame(z.loc['7mer'].values) # Using pd.DataFrame as it removes the descriptors like Name:, dtype:. Don't know how it exactly works
                    yu = pd.DataFrame(z.iloc[l])
                    proper_format_rbp = r2.columns[l]
                    for_rbp_scrip = z.iloc[l]
                if Cutoffs[l-1] < z.iloc[l].values:  # Over here looking for other strong motif matches for a given sequence for which the sequence's Z-score crossed the cutoff
                    Other_matches.append(pd.DataFrame(z.iloc[l]))
                l = l+1
            Rep_motifs.append(yu)
            colx.append(Sev_mer)
            All_Motifs.append(proper_format_rbp)
            Segul.append(proper_format_rbp)
            All_R_Motifs.append(for_rbp_scrip)
    #high = pd.Series([Rep_motifs, colx, Other_matches])
    #Cons.update({a: high})
    Cons2.update({a : All_Motifs})
    

'''
Put value equal to z for a general table and put high for just the highest z-score and corresponding motif.
df.to_csv('RepressorsCISBP_ZScores_Format2.csv')  Over format 2, I prefer format 3, but the code for format 3 has yet to be automated.
'''
df = pd.DataFrame.from_dict(Cons, orient="index")
df2 = pd.DataFrame.from_dict(Cons2, orient="index")
#df.to_csv('All_RepressorsCISBP_HighestZScores_Motifs.csv')

# Extracting the strongly matched motifs (for all repressors)


Rep_motif_matches = set((pd.DataFrame(All_R_Motifs).T).columns)


(df2.T).to_csv('Motifs_Repressors')

Unique_Motifs = set(Segul)

In [14]:
'''
Repeating the same for activators.
'''
Activators = pd.Series(['TCTGCACATG_17e','GACGAAGCAA_25e','TAATTCTAAG_44e','GAGGTAGTAA_50e','ACTAACTGCA_47e','TAAACAGAAC_69e','TTCAGCAGGT_77e','TCAAGGATGG_82e','CGGACAAAGT_17B','ACGCAAAAGG_25B','GAGAGACATG_44B','ATTACGAATG_50B','CTGCGGCTGA_47B','AGAGGCAAAT_69B','TATCGTTAGA_77B','GGCCCATCAA_82B'])
Cons2 = {}
Cons = {}
Segul = []
for a in Activators:
    colx = []
    Act_motifs = []
    All_A_Motifs = []
    Other_matches = []
    All_Motifs = []
    for c in r2['7mer']:
          if c in a:
            z = (r2.loc[r2[r2['7mer']==c].index.values]).T
            l=1
            maxval = 0
            while (l < 245): 
                if maxval < z.iloc[l].values:# Looking for the strongest motif match among all other motifs
                    maxval = z.iloc[l].values
                    Sev_mer= pd.DataFrame(z.loc['7mer'].values) # Using pd.DataFrame as it removes the descriptors like Name:, dtype:. Don't know how it exactly works
                    yu = pd.DataFrame(z.iloc[l])
                    for_rbp_scrip = z.iloc[l]
                    proper_format_rbp = r2.columns[l]
                if Cutoffs[l-1] < z.iloc[l].values:# Over here looking for other strong motif matches for a given sequence for which the sequence's Z-score crossed the cutoff
                    Other_matches.append(pd.DataFrame(z.iloc[l]))
                l = l+1
            Act_motifs.append(yu)
            colx.append(Sev_mer)
            All_A_Motifs.append(for_rbp_scrip)
            All_Motifs.append(proper_format_rbp)
            Segul.append(proper_format_rbp)
    high = pd.Series([Act_motifs, colx, Other_matches])
    #Cons.update({a: high})
    Cons2.update({a : All_Motifs})
                                  
            
df = pd.DataFrame.from_dict(Cons, orient="index")
#df.to_csv('All_ActivatorsCISBP_ZScores_Format2.csv')
#df.to_csv('All_ActivatorsCISBP_HighestZScores_Motifs.csv')

Unique_Motifs_A = set(Segul)

'\nThe following version results in a better format but has to be automated. \n'

In [21]:
'''
Ranking and sorting of Z-Scores (All elements - repressors and activators both cis-elements and barcodes).
'''
# Part1- Taking a chunk of bigger dataframe. Taking only the columns of the motifs which were found in my repressors and then sorting in descending order.
Unq = list(Unique_Motifs)
Ruck = list(Unique_Motifs_A)
Unq1 = (*Unq, *Ruck)
Unq2 = list(set(Unq1))
Unq2.append('7mer')
st = r2[Unq2]
sort_df = st.sort_values(by = Unq, ascending = False)
sort_df.reset_index(drop=True, inplace=True)
## Sorting the E-scores as well
st_E = r5[Unq2]
sort_Escore = st_E.sort_values(by = Unq, ascending = False)
sort_Escore.reset_index(drop =True, inplace =True)

# Part-2 Finding the 7-mers of interest and ranking.

feat2 = sort_df.describe()
UppQ_sort_df = feat2.loc['75%'] #This is similar to original dataframe's cutoffs

All_elements = [*Repressors, *Activators]

Ranking = {}

for a in All_elements:
    Rep_motifs = []
    for c in sort_df['7mer']:
        if c in a:
            z = (sort_df.loc[sort_df[sort_df['7mer']==c].index.values]).T
            l=0
            maxval = 0
            while (l < 32):
                if maxval < z.iloc[l].values:# Looking for the strongest motif match among all other motifs
                    maxval = z.iloc[l].values
                    yu = pd.DataFrame(z.iloc[l])
                l = l+1
            Rep_motifs.append(yu)
    Ranking.update({a : Rep_motifs}) 

Ranki = pd.DataFrame.from_dict(Ranking, orient = 'index')
#Ranki.to_csv('Ranked_Elements_ZScores.csv')



In [8]:
'''
Searching insignificant element (insignificant post Holm-Bonferroni corrections) barcode and cis-elements within the Z-score DataFrame. Barcode and cis-element IDs are appeneded with the respective elements.

'''
Insig_elements = pd.Series(['AACAACAACA_84B','CAAATATAGC_114B', 'GGAAAGTCAC_62B', 'TAGAAGCGGC_53B','TAATGGACTA_100B','GCCCAACGCA_12B', 'ATGGAATAAA_34B', 'ATTCGGTTTG_81B', 
'CAAGCCAAGA_90B',
'ATAAAGTGCA_102B',
'GCTTTGCGCT_22B',
'CATGGTAGGC_38B',
'AGTTACGCGG_76B',
'AGCCACTGGG_84e',
'ACCAGATCGA_114e',
'ACCTCCTCGC_62e',
'CCGTATGCAG_53e',
'AAACCCTAGT_100e',
'AGGCACGGC_12e',
'ACAATAGTGA_34e',
'GGTCAATCGA_81e',
'CGACTGTGAA_90e',
'AAATACTAAT_102e',
'TGATCGCTGC_22e',
'AACATGACGC_38e',
'CAACGAAGTG_76e'])


Cons = {}
Cons2 = {}
Segul = []
for a in Insig_elements:
    colx = []
    Rep_motifs = []
    All_R_Motifs = []
    All_Motifs = []
    Other_matches = []
    for c in r2['7mer']:
        if c in a:
            z = (r2.loc[r2[r2['7mer']==c].index.values]).T
            l=1
            maxval = 0
            while (l < 245):
                if maxval < z.iloc[l].values:# Looking for the strongest motif match among all other motifs
                    maxval = z.iloc[l].values
                    Sev_mer= pd.DataFrame(z.loc['7mer'].values) # Using pd.DataFrame as it removes the descriptors like Name:, dtype:. Don't know how it exactly works
                    yu = pd.DataFrame(z.iloc[l])
                    proper_format_rbp = r2.columns[l]
                    for_rbp_scrip = z.iloc[l]
                if Cutoffs[l-1] < z.iloc[l].values:  # Over here looking for other strong motif matches for a given sequence for which the sequence's Z-score crossed the cutoff
                    Other_matches.append(pd.DataFrame(z.iloc[l]))
                l = l+1
            Rep_motifs.append(yu)
            colx.append(Sev_mer)
            All_Motifs.append(proper_format_rbp)
            Segul.append(proper_format_rbp)
            All_R_Motifs.append(for_rbp_scrip)
    high = pd.Series([Rep_motifs, colx, Other_matches])
    Cons.update({a: high})
    Cons2.update({a : All_Motifs})
    

'''
Put value equal to z for a general table and put high for just the highest z-score and corresponding motif.
df.to_csv('Insig_elementsCISBP_ZScores_Format2.csv')  Over format 2, I prefer format 3, but the code for format 3 has yet to be automated.
'''
df = pd.DataFrame.from_dict(Cons, orient="index")
df2 = pd.DataFrame.from_dict(Cons2, orient="index")
df.to_csv('All_Insig_elementsCISBP_HighestZScores_Motifs.csv')

# Extracting the strongly matched motifs (for all Insig_elements)


Rep_motif_matches = set((pd.DataFrame(All_R_Motifs).T).columns)


(df2.T).to_csv('Motifs_Insig_elements')

Unique_Motifs = set(Segul)
'''
Ranking and sorting of Z-Scores (All elements - insiginificant cis-elements and barcodes).
'''
# Part1- Taking a chunk of bigger dataframe. Taking only the columns of the motifs which were found in my repressors and then sorting in descending order.
Unq2 = list(Unique_Motifs)
Unq2.append('7mer')
st = r2[Unq2]
sort_df = st.sort_values(by = Unq2, ascending = False)

sort_df.reset_index(drop=True, inplace=True)
## Sorting the E-scores as well
st_E = r5[Unq2]
sort_Escore = st_E.sort_values(by = Unq2, ascending = False)
sort_Escore.reset_index(drop =True, inplace =True)

# Part-2 Finding the 7-mers of interest and ranking.

feat2 = sort_df.describe()
UppQ_sort_df = feat2.loc['75%'] #This is similar to original dataframe's cutoffs


All_elements = ['AACAACAACA_84B','CAAATATAGC_114B', 'GGAAAGTCAC_62B', 'TAGAAGCGGC_53B','TAATGGACTA_100B','GCCCAACGCA_12B', 'ATGGAATAAA_34B', 'ATTCGGTTTG_81B', 
'CAAGCCAAGA_90B',
'ATAAAGTGCA_102B',
'GCTTTGCGCT_22B',
'CATGGTAGGC_38B',
'AGTTACGCGG_76B',
'AGCCACTGGG_84e',
'ACCAGATCGA_114e',
'ACCTCCTCGC_62e',
'CCGTATGCAG_53e',
'AAACCCTAGT_100e',
'AGGCACGGC_12e',
'ACAATAGTGA_34e',
'GGTCAATCGA_81e',
'CGACTGTGAA_90e',
'AAATACTAAT_102e',
'TGATCGCTGC_22e',
'AACATGACGC_38e',
'CAACGAAGTG_76e']
Ranking = {}

for a in All_elements:
    Rep_motifs = []
    for c in sort_df['7mer']:
        if c in a:
            z = (sort_df.loc[sort_df[sort_df['7mer']==c].index.values]).T
            l=0
            maxval = 0
            while (l < 13):
                if maxval < z.iloc[l].values:# Looking for the strongest motif match among all other motifs
                    maxval = z.iloc[l].values
                    yu = pd.DataFrame(z.iloc[l])
                l = l+1
            Rep_motifs.append(yu)
    Ranking.update({a : Rep_motifs}) 

Ranki = pd.DataFrame.from_dict(Ranking, orient = 'index')
Ranki.to_csv('Ranked_Insig_Elements_ZScores.csv')

In [5]:
'''
Searching insignificant element barcode and cis-elements within the Z-score DataFrame. Barcode and cis-element IDs are appeneded with the respective elements.

'''
Insig_elements = pd.Series(['CGTTATCCAG_6B',
'GTTCGGGCCG_10B',
'CTCCAAGAGC_13B',
'CTTAGTAACG_16B',
'ATACCAGGAA_19B',
'CGTGGGTGCA_24B',
'ATCTTATTGG_27B',
'ATGGATCGGT_31B',
'TTAGAAGTCG_32B',
'CTAAGATGCA_41B',
'GTCATCGCTA_42B',
'CGGTGCGTGT_43B',
'ACAGAAGAGA_45B',
'ATAATCGTGA_49B',
'ATCCAGGTGT_59B',
'TACTTTTGTA_64B',
'AGTTTCCGTG_66B',
'AAATAGATTC_71B',
'GATACGCTGG_78B',
'GGGGTAGATG_80B',
'TCTGAGATCT_83B',
'ATAGCGCGAT_95B',
'GTGTAAACTA_96B',
'CGCGATGAAA_97B',
'AGATAAAAAC_107B',
'TAAAACATAC_109B',
'GGTAAGGAAG_112B',
'GAGTGCCCCC_6e',
'ACAAATCTAA_10e',
'TAAGATTGAA_13e',
'ACTTATGGGG_16e',
'AGCACAGAAA_19e',
'AATCTTAAGT_24e',
'AGTGTTAAAA_27e',
'TAAGAATAAG_31e',
'ACTAGGATAA_32e',
'CCTATTTATT_41e',
'AAGAAAATAA_42e',
'GCATCTGTAA_43e',
'AATTGCCTAT_45e',
'GCCGCGAATG_49e',
'CTATAACCAC_59e',
'CAACTAAGCT_64e',
'CATTCCTTTG_66e',
'GTGGAATAAG_71e',
'ACTAGTAAGG_78e',
'AAGAGATCAA_80e',
'AGTCCTTGCC_83e',
'ACCACGCTGT_95e',
'TTACCAATGC_96e',
'TACCAACAAG_97e',
'CTATATACAA_107e',
'TCGTCCGAAT_109e',
'CTTTCAAATC_112e'])


Cons = {}
Cons2 = {}
Segul = []
for a in Insig_elements:
    colx = []
    Rep_motifs = []
    All_R_Motifs = []
    All_Motifs = []
    Other_matches = []
    for c in r2['7mer']:
        if c in a:
            z = (r2.loc[r2[r2['7mer']==c].index.values]).T
            l=1
            maxval = 0
            while (l < 245):
                if maxval < z.iloc[l].values:# Looking for the strongest motif match among all other motifs
                    maxval = z.iloc[l].values
                    Sev_mer= pd.DataFrame(z.loc['7mer'].values) # Using pd.DataFrame as it removes the descriptors like Name:, dtype:. Don't know how it exactly works
                    yu = pd.DataFrame(z.iloc[l])
                    proper_format_rbp = r2.columns[l]
                    for_rbp_scrip = z.iloc[l]
                if Cutoffs[l-1] < z.iloc[l].values:  # Over here looking for other strong motif matches for a given sequence for which the sequence's Z-score crossed the cutoff
                    Other_matches.append(pd.DataFrame(z.iloc[l]))
                l = l+1
            Rep_motifs.append(yu)
            colx.append(Sev_mer)
            All_Motifs.append(proper_format_rbp)
            Segul.append(proper_format_rbp)
            All_R_Motifs.append(for_rbp_scrip)
    high = pd.Series([Rep_motifs, colx, Other_matches])
    Cons.update({a: high})
    Cons2.update({a : All_Motifs})
    

'''
Put value equal to z for a general table and put high for just the highest z-score and corresponding motif.
df.to_csv('Insig_elementsCISBP_ZScores_Format2.csv')  Over format 2, I prefer format 3, but the code for format 3 has yet to be automated.
'''
df = pd.DataFrame.from_dict(Cons, orient="index")
df2 = pd.DataFrame.from_dict(Cons2, orient="index")
df.to_csv('2All_Insig_elementsCISBP_HighestZScores_Motifs.csv')

# Extracting the strongly matched motifs (for all Insig_elements)


Rep_motif_matches = set((pd.DataFrame(All_R_Motifs).T).columns)


(df2.T).to_csv('2Motifs_Insig_elements')

Unique_Motifs = set(Segul)
'''
Ranking and sorting of Z-Scores (All elements - insiginificant cis-elements and barcodes).
'''
# Part1- Taking a chunk of bigger dataframe. Taking only the columns of the motifs which were found in my repressors and then sorting in descending order.
Unq2 = list(Unique_Motifs)
Unq2.append('7mer')
st = r2[Unq2]
sort_df = st.sort_values(by = Unq2, ascending = False)

sort_df.reset_index(drop=True, inplace=True)
## Sorting the E-scores as well
st_E = r5[Unq2]
sort_Escore = st_E.sort_values(by = Unq2, ascending = False)
sort_Escore.reset_index(drop =True, inplace =True)

# Part-2 Finding the 7-mers of interest and ranking.

feat2 = sort_df.describe()
UppQ_sort_df = feat2.loc['75%'] #This is similar to original dataframe's cutoffs


All_elements = ['CGTTATCCAG_6B',
'GTTCGGGCCG_10B',
'CTCCAAGAGC_13B',
'CTTAGTAACG_16B',
'ATACCAGGAA_19B',
'CGTGGGTGCA_24B',
'ATCTTATTGG_27B',
'ATGGATCGGT_31B',
'TTAGAAGTCG_32B',
'CTAAGATGCA_41B',
'GTCATCGCTA_42B',
'CGGTGCGTGT_43B',
'ACAGAAGAGA_45B',
'ATAATCGTGA_49B',
'ATCCAGGTGT_59B',
'TACTTTTGTA_64B',
'AGTTTCCGTG_66B',
'AAATAGATTC_71B',
'GATACGCTGG_78B',
'GGGGTAGATG_80B',
'TCTGAGATCT_83B',
'ATAGCGCGAT_95B',
'GTGTAAACTA_96B',
'CGCGATGAAA_97B',
'AGATAAAAAC_107B',
'TAAAACATAC_109B',
'GGTAAGGAAG_112B',
'GAGTGCCCCC_6e',
'ACAAATCTAA_10e',
'TAAGATTGAA_13e',
'ACTTATGGGG_16e',
'AGCACAGAAA_19e',
'AATCTTAAGT_24e',
'AGTGTTAAAA_27e',
'TAAGAATAAG_31e',
'ACTAGGATAA_32e',
'CCTATTTATT_41e',
'AAGAAAATAA_42e',
'GCATCTGTAA_43e',
'AATTGCCTAT_45e',
'GCCGCGAATG_49e',
'CTATAACCAC_59e',
'CAACTAAGCT_64e',
'CATTCCTTTG_66e',
'GTGGAATAAG_71e',
'ACTAGTAAGG_78e',
'AAGAGATCAA_80e',
'AGTCCTTGCC_83e',
'ACCACGCTGT_95e',
'TTACCAATGC_96e',
'TACCAACAAG_97e',
'CTATATACAA_107e',
'TCGTCCGAAT_109e',
'CTTTCAAATC_112e']
Ranking = {}

for a in All_elements:
    Rep_motifs = []
    for c in sort_df['7mer']:
        if c in a:
            z = (sort_df.loc[sort_df[sort_df['7mer']==c].index.values]).T
            l=0
            maxval = 0
            while (l < 14):
                if maxval < z.iloc[l].values:# Looking for the strongest motif match among all other motifs
                    maxval = z.iloc[l].values
                    yu = pd.DataFrame(z.iloc[l])
                l = l+1
            Rep_motifs.append(yu)
    Ranking.update({a : Rep_motifs}) 

Ranki = pd.DataFrame.from_dict(Ranking, orient = 'index')
Ranki.to_csv('2Ranked_Insig_Elements_ZScores.csv')

In [2]:
'''
Repeating the same for entire minigene.
'''
neuronal = "ATGGATTACAAGGATGACGATGACAAGGGGGTACCTGCCCCAAAAAAAAAACGCAAAGTGGAGGACCCAGTACCCGGATCTGAATTCATATATGCTAGCAGTATCAGTGAGGTGGCACCTGTCGATTCGTCTGCGCCTGTTAAAGGTAAAAATGTTTGAAAAATTTCTATAGATCAAATATTAAAACCACCCGGACGCGAGATTTTTCCTTTTTTTTTGTCCAAAAATCGGTCTCGACACGACAATTTTCGTTATATGCAAACGGATGTGCACCTTTAAAGAGTACTGTAAATTAACAAATCGTGAGATAAACTATGAGAAATCGATGAAAATTCCACATCAATGAAACTTTTATATTACAGTACTCTTTAAAGCCACACGCCGAAAAAATTGTAGCGTCGAGACCGGATACCGTATAGACCAAAAATCGCAAAATTTCGCGCCTGGCTAATAAAACACGTTTTTCATTGAGCTGAAATTTGAAAAATTTCGGAAATTTATTAGCAATAAATTTTTTAGCTAAAACTAAACTCTCGCGAGTTTCGGAAAATGTTGTGCCGGATGAGACGCCAAAATTGATTCCACTGGAGAAATTGAAAAAAGGTGATCCGCAGAGACATGGTTTCAAATTGGNNNNNNNNNNTGcTaTGTCGTTTTTcAAaccTCTTTACAAACGAAAAATCTCACTATTTGGTTCTAGTCGTATAATTCTTAGTTTTATGATTTTCATCCACAAACTTTCAATTTAATTCAAATTTCCATAACTGTCCAATTTTCTTTTAAAAAACTGATTTTAAATGATAGTTTCGGAAAATGTTGTGCCGGATGAGACGCCAAAATTGATTCAGTTTCTGCTTCTGCACTTCTATATGCTCAATCGGTTCTGTCTCTTATAACTCAATTCAAATTTTTTCAGAAGATCCATTACCTCCACCTGCAAACCGCCCGGGTGGTCGTGCAGCTCTTCCACCACACATTCGTTAANNNNNNNNNNTACGTACTTCTGAGTCCAATTACTCTTCAACATCCCTACATGCTCTTTCTCCCTGTGCTCCCACCCCCTATTTTTGTTATTATCAAAAAACTTCTCTTAATTTCTTTGTTTTTTAGCTTCTTTTAAGCCACCTCTAACAATGAAATTGTGTAGATTCAAAAATAGAATTAATTCGTAATAAAAAGTCGA"
Cons2 = {}
Cons = {}
Segul = []

colx = []
Act_motifs = []
All_A_Motifs = []
Other_matches = []
All_Motifs = []
print(len(neuronal))
for c in r2['7mer']:
    pos = 0
    if c in neuronal:
        z = (r2.loc[r2[r2['7mer']==c].index.values]).T
        pos = neuronal.index(c)
        l=1
        maxval = 0
        while (l < 245): 
            if maxval < z.iloc[l].values:# Looking for the strongest motif match among all other motifs
                maxval = z.iloc[l].values
                Sev_mer= pd.DataFrame(z.loc['7mer'].values) # Using pd.DataFrame as it removes the descriptors like Name:, dtype:. Don't know how it exactly works
                yu = pd.DataFrame(z.iloc[l])
                for_rbp_scrip = z.iloc[l]
                proper_format_rbp = r2.columns[l]
            if Cutoffs[l-1] < z.iloc[l].values:# Over here looking for other strong motif matches for a given sequence for which the sequence's Z-score crossed the cutoff
                Other_matches.append(pd.DataFrame(z.iloc[l]))
            l = l+1
        Act_motifs.append(yu)
        colx.append(Sev_mer)
        All_A_Motifs.append(for_rbp_scrip)
        All_Motifs.append(proper_format_rbp)
        Segul.append(proper_format_rbp)
    high = pd.Series([Act_motifs, colx, Other_matches])
    Cons.update({pos: high})
    #Cons2.update({pos : All_Motifs})
                                  
            
df = pd.DataFrame.from_dict(Cons, orient="index")
#df.to_csv('All_ActivatorsCISBP_ZScores_Format2.csv')
df.to_csv('Minigene_ZScores_Motifs.csv')

#Unique_Motifs_A = set(Segul)


1183


KeyboardInterrupt: 

'\nThe following version results in a better format but has to be automated. \n'

In [3]:
'''
Repeating the same for entire minigene.
'''
neuronal = "ATGGATTACAAGGATGACGATGACAAGGGGGTACCTGCCCCAAAAAAAAAACGCAAAGTGGAGGACCCAGTACCCGGATCTGAATTCATATATGCTAGCAGTATCAGTGAGGTGGCACCTGTCGATTCGTCTGCGCCTGTTAAAGGTAAAAATGTTTGAAAAATTTCTATAGATCAAATATTAAAACCACCCGGACGCGAGATTTTTCCTTTTTTTTTGTCCAAAAATCGGTCTCGACACGACAATTTTCGTTATATGCAAACGGATGTGCACCTTTAAAGAGTACTGTAAATTAACAAATCGTGAGATAAACTATGAGAAATCGATGAAAATTCCACATCAATGAAACTTTTATATTACAGTACTCTTTAAAGCCACACGCCGAAAAAATTGTAGCGTCGAGACCGGATACCGTATAGACCAAAAATCGCAAAATTTCGCGCCTGGCTAATAAAACACGTTTTTCATTGAGCTGAAATTTGAAAAATTTCGGAAATTTATTAGCAATAAATTTTTTAGCTAAAACTAAACTCTCGCGAGTTTCGGAAAATGTTGTGCCGGATGAGACGCCAAAATTGATTCCACTGGAGAAATTGAAAAAAGGTGATCCGCAGAGACATGGTTTCAAATTGGNNNNNNNNNNTGcTaTGTCGTTTTTcAAaccTCTTTACAAACGAAAAATCTCACTATTTGGTTCTAGTCGTATAATTCTTAGTTTTATGATTTTCATCCACAAACTTTCAATTTAATTCAAATTTCCATAACTGTCCAATTTTCTTTTAAAAAACTGATTTTAAATGATAGTTTCGGAAAATGTTGTGCCGGATGAGACGCCAAAATTGATTCAGTTTCTGCTTCTGCACTTCTATATGCTCAATCGGTTCTGTCTCTTATAACTCAATTCAAATTTTTTCAGAAGATCCATTACCTCCACCTGCAAACCGCCCGGGTGGTCGTGCAGCTCTTCCACCACACATTCGTTAANNNNNNNNNNTACGTACTTCTGAGTCCAATTACTCTTCAACATCCCTACATGCTCTTTCTCCCTGTGCTCCCACCCCCTATTTTTGTTATTATCAAAAAACTTCTCTTAATTTCTTTGTTTTTTAGCTTCTTTTAAGCCACCTCTAACAATGAAATTGTGTAGATTCAAAAATAGAATTAATTCGTAATAAAAAGTCGA"
Cons2 = {}
Cons = {}
Segul = []

colx = []
Act_motifs = []
All_A_Motifs = []
Other_matches = []
All_Motifs = []
print(len(neuronal))
for c in r2['7mer']:
    pos = 0
    if c in neuronal:
        z = (r2.loc[r2[r2['7mer']==c].index.values]).T
        pos = neuronal.index(c)
        l=1
        maxval = 0
        while (l < 245): 
            if maxval < z.iloc[l].values:# Looking for the strongest motif match among all other motifs
                maxval = z.iloc[l].values
                Sev_mer= pd.DataFrame(z.loc['7mer'].values) # Using pd.DataFrame as it removes the descriptors like Name:, dtype:. Don't know how it exactly works
                yu = pd.DataFrame(z.iloc[l])
                for_rbp_scrip = z.iloc[l]
                proper_format_rbp = r2.columns[l]
            #if Cutoffs[l-1] < z.iloc[l].values:# Over here looking for other strong motif matches for a given sequence for which the sequence's Z-score crossed the cutoff
                #Other_matches.append(pd.DataFrame(z.iloc[l]))
            l = l+1
        Act_motifs.append(yu)
        colx.append(Sev_mer)
        All_A_Motifs.append(for_rbp_scrip)
        All_Motifs.append(proper_format_rbp)
        Segul.append(proper_format_rbp)
    high = pd.Series([Act_motifs, colx])
    Cons.update({pos: high})
    #Cons2.update({pos : All_Motifs})
                                  
            
df = pd.DataFrame.from_dict(Cons, orient="index")
#df.to_csv('All_ActivatorsCISBP_ZScores_Format2.csv')
df.to_csv('2Minigene_ZScores_Motifs.csv')

#Unique_Motifs_A = set(Segul)


1183


In [ ]:
'''
RBP analysis below

'''

In [16]:
ru = {}
for gigi in Unique_Motifs_A:
    hichki2 = []
    for bella in Cons2:
        if gigi in Cons2.get(bella):
            hichki2.append(str(bella))
    ru.update({gigi: str(hichki2)})


df_fi = pd.DataFrame.from_dict(ru, orient='index')

In [9]:
Trial = rbp_info[['Motif_ID', 'RBP_Name', 'RBP_Species', 'RBP_Status','Family_Name']]
#hichki = ['M125_0.6', 'M042_0.6', 'M200_0.6', 'M105_0.6', 'M081_0.6', 'M082_0.6', 'M128_0.6', 'M107_0.6', 'M057_0.6', 'M087_0.6', 'M212_0.6', 'M008_0.6', 'M109_0.6', 'M147_0.6', 'M006_0.6', 'M121_0.6', 'M019_0.6', 'M026_0.6', 'M037_0.6', 'M027_0.6', 'M094_0.6', 'M169_0.6']
mask = (Trial['Motif_ID']).isin(Unique_Motifs_A)
Trial = Trial[mask]
Trial.reset_index(drop = True, inplace = True)

#col_one_list = Trial['Motif_ID'].to_list()
#new_df = pd.DataFrame(TO_Merge[['RBP_Name', 'RBP_Species', 'RBP_Status','Family_Name']].values, index = col_one_list, columns = ['RBP_Name', 'RBP_Species', 'RBP_Status','Family_Name'])

Trial['Elements_mapped'] = Trial['Motif_ID'].map(ru)

#(Trial).to_csv("Act_RBP_Motifs.csv")


In [92]:
'''
Alternative method using json

import json
Cons = {}
io = []
i = 0 
for a in Repressors:
    for c in r2['7mer']:
          if c in a:
                z = (r2.loc[r2[r2['7mer']==c].index.values]).to_json()
                Cons.update({a: z})

df = pd.DataFrame.from_dict(Cons, orient="index")
df.to_csv('Repressors_CISBP_ZScores.csv')
'''


'\nAlternative method using json\n\nimport json\nCons = {}\nio = []\ni = 0 \nfor a in Repressors:\n    for c in r2[\'7mer\']:\n          if c in a:\n                z = (r2.loc[r2[r2[\'7mer\']==c].index.values]).to_json()\n                Cons.update({a: z})\n\ndf = pd.DataFrame.from_dict(Cons, orient="index")\ndf.to_csv(\'Repressors_CISBP_ZScores.csv\')\n'

In [17]:
rbp_extract = {}
Act_motif_matches = set((pd.DataFrame(Act_motifs).T).columns)#Extracting the strongly matched motifs for all activators.

for a in Act_motif_matches:
    for jam in rbp_info['Motif_ID']:
         if a in jam: 
                herc = (rbp_info.loc[rbp_info[rbp_info['Motif_ID']==jam].index.values])
                herc_d = dict(herc)
                rbp_extract.update({a : herc_d})
        
hpe = pd.DataFrame.from_dict(rbp_extract, orient='index')
#(['MSource_Author','MSource_Year','PMID','MSource_Version','RBPSource_Name','RBPSource_URL','RBPSource_Year','RBPSource_Month','RBPSource_Day'])
#(hpe).to_csv("CISBP_RBP_Extract.csv")  


In [127]:
array= []
for unio in Rep_motif_matches:
        for iop in rbp_info['Motif_ID']:
            if unio == iop:
                rbp = (rbp_info.loc[rbp_info[rbp_info['Motif_ID']==iop].index.values])
                array.append(rbp.index)
(rbp_info.reindex([170,192,174,193,227,47,75,78,146,177,67,183,150,208,117,205,116,171,112,3,20])).to_csv('Repressor binding RBPs')

In [128]:
array2= []
for unio in Act_motif_matches:
        for iop in rbp_info['Motif_ID']:
            if unio == iop:
                rbp = (rbp_info.loc[rbp_info[rbp_info['Motif_ID']==iop].index.values])
                array2.append(rbp.index)
    
                
(rbp_info.reindex([170,192,174,193,227,47,227, 75,78, 177,67,150,208,183,116,171,3,20])).to_csv('Activator binding RBPs')


In [22]:
#import sys
#sys.path.append('/Users/sanjanabhatnagar/opt/anaconda3/envs/tf2/lib/python3.7/site-packages/pandas')
import pandas as pd

KC1 = pd.ExcelFile('Desktop/Clusters.xlsx')
pd.read_excel(KC1, 'Sheet1')
Kclust = pd.DataFrame(ru)
Kclust = Kclust.sort_values(by = 'Cluster_ID')
Kclust.to_csv('Clusters_Els.csv')


In [ ]:
'''
E-score analysis below

'''

In [2]:
'''
Searching repressor barcode and cis-elements within the Z-score DataFrame. Barcode and cis-element IDs are appeneded with the respective elements.

'''
import pandas as pd

rc = pd.read_csv("Desktop/PhD project/CISBP_results/entiredata_2020_04_14_1-09_pm/Escores.txt", sep = "\t")
r5 = pd.DataFrame(rc)

Repressors = pd.Series(['CGAGGAGTCC_9e','ATCATGGCGC_11e','CAGATAACGA_15e','ACCATCCTGA_20e','CATAGATCGC_26e','CATTCAGCCC_28e','CACAGAAGGC_33e','CAAAATAAAC_37e','AACATGACGC_38e','AATCACCAAC_39e','CAATTGCGAT_40e','CAAATGAGAA_52e','CAGATAAACA_54e','AGACTAATTA_56e','CGGGGCTCAA_60e','AAAAATGCGA_61e','ACCTCCTCGC_62e','CGCACAGATG_68e','GCGCTACCCG_103e','GCAACACCTC_104e','GCGGTCACGT_110e','GCCAGTTTCT_9B','TGCCACTCAT_11B','TAAAACACTT_15B','CGATGAGAAG_20B','TAATTCAGTA_26B','ATCAGACCCA_28B','GAGTGAGGAA_33B','ACGCGCGGGC_37B','CATGGTAGGC_38B','TAATGGCGTA_39B','TTTACCTAGC_40B','AACAAGCAGC_52B','TTGACGGCGC_54B','AAGGGCCTCG_56B','AGAAGGATAT_60B','GAGAAGGCAG_61B','GGAAAGTCAC_62B','GGGTTCGAGA_68B','ATGATGTACG_103B','TGTCGCATCC_104B','ACAACAAAGG_110B'])


Cons = {}
Cons2 = {}
Segul = []

for a in Repressors:
    colx = []
    Rep_motifs = []
    All_R_Motifs = []
    All_Motifs = []
    for c in r5['7mer']:
        if c in a:
            z = (r5.loc[r5[r5['7mer']==c].index.values]).T
            l=1
            while (l < 245):
                if  ((z.iloc[l].values > 0.45) or (z.iloc[l].values == 0.45)):# Looking for the strongest motif match among all other motifs
                    Rep_motifs.append(pd.DataFrame(z.iloc[l]))
                    colx.append(pd.DataFrame(z.loc['7mer'].values))
                    All_Motifs.append(r5.columns[l])
                    Segul.append(r5.columns[l])
                    All_R_Motifs.append(z.iloc[l])
                l = l+1
    #CQ_low = pd.Series([CQ_Rep_motifs, CQ_colx])        
    high = pd.Series([Rep_motifs, colx])
    Cons.update({a: high})
    #Cons_l.update({a: CQ_low})
    Cons2.update({a : All_Motifs})
    

'''
Put value equal to z for a general table and put high for just the highest z-score and corresponding motif.
df.to_csv('RepressorsCISBP_ZScores_Format2.csv')  Over format 2, I prefer format 3, but the code for format 3 has yet to be automated.
'''
df = pd.DataFrame.from_dict(Cons, orient="index")
df2 = pd.DataFrame.from_dict(Cons2, orient="index")
#df.to_csv('All_RepressorsCISBP_ZScores_Format2.csv') #Once the Format3 part would be automated, this format won't be required
#df.to_csv('EScores_RepressorsCISBPMotifs.csv')

# Extracting the strongly matched motifs (for all repressors)


In [15]:
'''
Repeating the same for activators.
'''
Activators = pd.Series(['TCTGCACATG_17e','GACGAAGCAA_25e','TAATTCTAAG_44e','GAGGTAGTAA_50e','ACTAACTGCA_47e','TAAACAGAAC_69e','TTCAGCAGGT_77e','TCAAGGATGG_82e','CGGACAAAGT_17B','ACGCAAAAGG_25B','GAGAGACATG_44B','ATTACGAATG_50B','CTGCGGCTGA_47B','AGAGGCAAAT_69B','TATCGTTAGA_77B','GGCCCATCAA_82B'])
Cons = {}

Segul = []
for a in Activators:
    colx = []
    Act_motifs = []
    All_A_Motifs = []
    Other_matches = []
    All_Motifs = []
    for c in r5['7mer']:
          if c in a:
            z = (r5.loc[r5[r5['7mer']==c].index.values]).T
            l=1
            maxval = 0
            while (l < 245): 
                if  ((z.iloc[l].values > 0.45) or (z.iloc[l].values == 0.45)):# Looking for the strongest motif match among all other motifs
                    Act_motifs.append(pd.DataFrame(z.iloc[l]))
                    colx.append(pd.DataFrame(z.loc['7mer'].values))
                    All_Motifs.append(r5.columns[l])
                    Segul.append(r5.columns[l])
                    All_A_Motifs.append(z.iloc[l])
                l = l+1

    high = pd.Series([Act_motifs, colx, Other_matches])
    Cons.update({a: high})
    Cons2.update({a : All_Motifs})
                                  


df = pd.DataFrame.from_dict(Cons, orient="index")
#df.to_csv('EScores_ActivatorsCISBPMotifs.csv')

'\nThe following version results in a better format but has to be automated. \n'

In [6]:
'''
Searching Insigressor barcode and cis-elements within the Z-score DataFrame. Barcode and cis-element IDs are appeneded with the respective elements.

'''
import pandas as pd

rc = pd.read_csv("Desktop/PhD project/CISBP_results/entiredata_2020_04_14_1-09_pm/Escores.txt", sep = "\t")
r5 = pd.DataFrame(rc)

Insig_els = pd.Series(['CGTTATCCAG_6B',
'GTTCGGGCCG_10B',
'CTCCAAGAGC_13B',
'CTTAGTAACG_16B',
'ATACCAGGAA_19B',
'CGTGGGTGCA_24B',
'ATCTTATTGG_27B',
'ATGGATCGGT_31B',
'TTAGAAGTCG_32B',
'CTAAGATGCA_41B',
'GTCATCGCTA_42B',
'CGGTGCGTGT_43B',
'ACAGAAGAGA_45B',
'ATAATCGTGA_49B',
'ATCCAGGTGT_59B',
'TACTTTTGTA_64B',
'AGTTTCCGTG_66B',
'AAATAGATTC_71B',
'GATACGCTGG_78B',
'GGGGTAGATG_80B',
'TCTGAGATCT_83B',
'ATAGCGCGAT_95B',
'GTGTAAACTA_96B',
'CGCGATGAAA_97B',
'AGATAAAAAC_107B',
'TAAAACATAC_109B',
'GGTAAGGAAG_112B',
'GAGTGCCCCC_6e',
'ACAAATCTAA_10e',
'TAAGATTGAA_13e',
'ACTTATGGGG_16e',
'AGCACAGAAA_19e',
'AATCTTAAGT_24e',
'AGTGTTAAAA_27e',
'TAAGAATAAG_31e',
'ACTAGGATAA_32e',
'CCTATTTATT_41e',
'AAGAAAATAA_42e',
'GCATCTGTAA_43e',
'AATTGCCTAT_45e',
'GCCGCGAATG_49e',
'CTATAACCAC_59e',
'CAACTAAGCT_64e',
'CATTCCTTTG_66e',
'GTGGAATAAG_71e',
'ACTAGTAAGG_78e',
'AAGAGATCAA_80e',
'AGTCCTTGCC_83e',
'ACCACGCTGT_95e',
'TTACCAATGC_96e',
'TACCAACAAG_97e',
'CTATATACAA_107e',
'TCGTCCGAAT_109e',
'CTTTCAAATC_112e','AACAACAACA_84B','CAAATATAGC_114B', 'GGAAAGTCAC_62B', 'TAGAAGCGGC_53B','TAATGGACTA_100B','GCCCAACGCA_12B', 'ATGGAATAAA_34B', 'ATTCGGTTTG_81B', 
'CAAGCCAAGA_90B',
'ATAAAGTGCA_102B',
'GCTTTGCGCT_22B',
'CATGGTAGGC_38B',
'AGTTACGCGG_76B',
'AGCCACTGGG_84e',
'ACCAGATCGA_114e',
'ACCTCCTCGC_62e',
'CCGTATGCAG_53e',
'AAACCCTAGT_100e',
'AGGCACGGC_12e',
'ACAATAGTGA_34e',
'GGTCAATCGA_81e',
'CGACTGTGAA_90e',
'AAATACTAAT_102e',
'TGATCGCTGC_22e',
'AACATGACGC_38e',
'CAACGAAGTG_76e'])

Cons = {}
Cons2 = {}
Segul = []

for a in Insig_els:
    colx = []
    Insig_motifs = []
    All_R_Motifs = []
    All_Motifs = []
    for c in r5['7mer']:
        if c in a:
            z = (r5.loc[r5[r5['7mer']==c].index.values]).T
            l=1
            while (l < 245):
                if  ((z.iloc[l].values > 0.45) or (z.iloc[l].values == 0.45)):# Looking for the strongest motif match among all other motifs
                    Insig_motifs.append(pd.DataFrame(z.iloc[l]))
                    colx.append(pd.DataFrame(z.loc['7mer'].values))
                    All_Motifs.append(r5.columns[l])
                    Segul.append(r5.columns[l])
                    All_R_Motifs.append(z.iloc[l])
                l = l+1
    #CQ_low = pd.Series([CQ_Insig_motifs, CQ_colx])        
    high = pd.Series([Insig_motifs, colx])
    Cons.update({a: high})
    #Cons_l.update({a: CQ_low})
    Cons2.update({a : All_Motifs})
    
df = pd.DataFrame.from_dict(Cons, orient="index")

df.to_csv('EScores_Insig_elsCISBPMotifs.csv')

# Extracting the strongly matched motifs (for all Insig_els)


In [ ]:
r5.describe()

In [ ]:
'''
Experimenting with stuff. Rough code.
'''
#hpe.drop(['MSource_Identifier','MSource_Type', 'MSource_Author', 'MSource_Year', 'PMID',
       #'MSource_Version', 'RBPSource_Name', 'RBPSource_URL', 'RBPSource_Year',
       #'RBPSource_Month', 'RBPSource_Day'], axis= 'columns') # For some reason this drop function isn't working.

gre = hpe.drop(['MSource_Identifier',
       'MSource_Type', 'MSource_Author', 'MSource_Year', 'PMID',
       'MSource_Version', 'RBPSource_Name', 'RBPSource_URL', 'RBPSource_Year',
       'RBPSource_Month', 'RBPSource_Day'], axis = 'columns')

#gre.to_csv("CISBP_RBP_Concise.csv")

maxzscore = erty.max(axis=1)
maxzscore[17]

'''
The following version results in a better format but has to be automated. 
'''

#ty = pd.Index(['TCTGCACATG_17e','GACGAAGCAA_25e','TAATTCTAAG_44e','GAGGTAGTAA_50e','ACTAACTGCA_47e','TAAACAGAAC_69e','TTCAGCAGGT_77e','TCAAGGATGG_82e','CGGACAAAGT_17B','ACGCAAAAGG_25B','GAGAGACATG_44B','ATTACGAATG_50B','CTGCGGCTGA_47B','AGAGGCAAAT_69B','TATCGTTAGA_77B','GGCCCATCAA_82B'])
#ry = r5.reindex([6306, 4618,4417,9287,9232,17,4629,9833,4242,8613,677,2088,8713,4160])

#vb = pd.Series(['CGAGGAGTCC_9e', 'CACAGAAGGC_33e', 'AATCACCAAC_39e', 'CGCACAGATG_68e', 'GCAACACCTC_104e', 'TAAAACACTT_15B', 'ATCAGACCCA_28B', 'ACGCGCGGGC_37B', 'AACAAGCAGC_52B', 'TTGACGGCGC_54B', 'AAGGGCCTCG_56B', 'AGAAGGATAT_60B', 'GAGAAGGCAG_61B', 'ACAACAAAGG_110B'], index=[6306, 4618,4417,9287,9232,17,4629,9833,4242,8613,677,2088,8713,4160])
#ry['Bar_Cis'] = vb

#cols = ['Bar_Cis','7mer', 'M001_0.6','M002_0.6','M003_0.6','M004_0.6','M005_0.6','M006_0.6','M007_0.6','M008_0.6','M009_0.6','M010_0.6','M011_0.6','M012_0.6','M013_0.6','M014_0.6','M015_0.6','M016_0.6','M017_0.6','M018_0.6','M019_0.6','M020_0.6','M021_0.6','M022_0.6','M023_0.6','M024_0.6','M025_0.6','M026_0.6','M027_0.6','M028_0.6','M029_0.6','M030_0.6','M031_0.6','M032_0.6','M033_0.6','M034_0.6','M035_0.6','M036_0.6','M037_0.6','M038_0.6','M039_0.6','M040_0.6','M041_0.6','M042_0.6','M043_0.6','M044_0.6','M045_0.6','M046_0.6','M047_0.6','M048_0.6','M049_0.6','M050_0.6','M051_0.6','M052_0.6','M053_0.6','M054_0.6','M055_0.6','M056_0.6','M057_0.6','M058_0.6','M059_0.6','M060_0.6','M061_0.6','M062_0.6','M063_0.6','M064_0.6','M065_0.6','M066_0.6','M067_0.6','M068_0.6','M069_0.6','M070_0.6','M071_0.6','M072_0.6','M073_0.6','M074_0.6','M075_0.6','M076_0.6','M077_0.6','M078_0.6','M079_0.6','M080_0.6','M081_0.6','M082_0.6','M083_0.6','M084_0.6','M085_0.6','M086_0.6','M087_0.6','M088_0.6','M089_0.6','M090_0.6','M091_0.6','M092_0.6','M093_0.6','M094_0.6','M095_0.6','M096_0.6','M097_0.6','M098_0.6','M099_0.6','M100_0.6','M101_0.6','M102_0.6','M103_0.6','M104_0.6','M105_0.6','M106_0.6','M107_0.6','M108_0.6','M109_0.6','M110_0.6','M111_0.6','M112_0.6','M113_0.6','M114_0.6','M115_0.6','M116_0.6','M117_0.6','M118_0.6','M119_0.6','M120_0.6','M121_0.6','M122_0.6','M123_0.6','M124_0.6','M125_0.6','M126_0.6','M127_0.6','M128_0.6','M129_0.6','M130_0.6','M131_0.6','M132_0.6','M133_0.6','M134_0.6','M135_0.6','M136_0.6','M137_0.6','M138_0.6','M139_0.6','M140_0.6','M141_0.6','M142_0.6','M143_0.6','M144_0.6','M145_0.6','M146_0.6','M147_0.6','M148_0.6','M149_0.6','M150_0.6','M151_0.6','M152_0.6','M153_0.6','M154_0.6','M155_0.6','M156_0.6','M157_0.6','M158_0.6','M159_0.6','M160_0.6','M161_0.6','M162_0.6','M163_0.6','M164_0.6','M165_0.6','M166_0.6','M167_0.6','M168_0.6','M169_0.6','M170_0.6','M171_0.6','M172_0.6','M173_0.6','M174_0.6','M175_0.6','M176_0.6','M177_0.6','M178_0.6','M179_0.6','M180_0.6','M181_0.6','M182_0.6','M183_0.6','M184_0.6','M185_0.6','M186_0.6','M187_0.6','M188_0.6','M189_0.6','M190_0.6','M191_0.6','M192_0.6','M193_0.6','M194_0.6','M195_0.6','M196_0.6','M197_0.6','M198_0.6','M199_0.6','M200_0.6','M201_0.6','M202_0.6','M203_0.6','M204_0.6','M205_0.6','M206_0.6','M207_0.6','M208_0.6','M209_0.6','M210_0.6','M211_0.6','M212_0.6','M213_0.6','M214_0.6','M215_0.6','M216_0.6','M217_0.6','M218_0.6','M219_0.6','M220_0.6','M221_0.6','M222_0.6','M223_0.6','M224_0.6','M225_0.6','M226_0.6','M227_0.6','M228_0.6','M229_0.6','M230_0.6','M231_0.6','M232_0.6','M233_0.6','M234_0.6','M235_0.6','M236_0.6','M237_0.6','M238_0.6','M239_0.6','M240_0.6','M241_0.6','M242_0.6','M243_0.6','M244_0.6']

#erty = ry.reindex(columns=cols)
#erty
#erty.to_csv('REP-CISBP-ZScores_Format3.csv')

#Rep_motif_matches = set((pd.DataFrame(All_R_Motifs).T).columns)
#new_dict = {}
#for key, value in Cons2.items(): 
    #if value in new_dict: 
       # new_dict[value].append(key) 
    #else: 
        #new_dict[value]=[key] 
        
#for i in new_dict: 
   # print(i, " :", new_dict[i])

#(df2.T).to_csv('Motifs_Repressors')

#Unique_Motifs = set(Segul)
#print(Unique_Motifs)

'''
The following version results in a better format but has to be automated. 
'''
#vb2 = r5.reindex([8577, 1153, 4682, 10302, 9217, 8736, 8847])
#ku = pd.Series(['GACGAAGCAA_25e','TAAACAGAAC_69e','TTCAGCAGGT_77e','CGGACAAAGT_17B','ACGCAAAAGG_25B','GAGAGACATG_44B','AGAGGCAAAT_69B'], index = [8577, 1153, 4682, 10302, 9217, 8736, 8847])

#vb2['Bar_Cis'] = ku

#cols = ['Bar_Cis','7mer', 'M001_0.6','M002_0.6','M003_0.6','M004_0.6','M005_0.6','M006_0.6','M007_0.6','M008_0.6','M009_0.6','M010_0.6','M011_0.6','M012_0.6','M013_0.6','M014_0.6','M015_0.6','M016_0.6','M017_0.6','M018_0.6','M019_0.6','M020_0.6','M021_0.6','M022_0.6','M023_0.6','M024_0.6','M025_0.6','M026_0.6','M027_0.6','M028_0.6','M029_0.6','M030_0.6','M031_0.6','M032_0.6','M033_0.6','M034_0.6','M035_0.6','M036_0.6','M037_0.6','M038_0.6','M039_0.6','M040_0.6','M041_0.6','M042_0.6','M043_0.6','M044_0.6','M045_0.6','M046_0.6','M047_0.6','M048_0.6','M049_0.6','M050_0.6','M051_0.6','M052_0.6','M053_0.6','M054_0.6','M055_0.6','M056_0.6','M057_0.6','M058_0.6','M059_0.6','M060_0.6','M061_0.6','M062_0.6','M063_0.6','M064_0.6','M065_0.6','M066_0.6','M067_0.6','M068_0.6','M069_0.6','M070_0.6','M071_0.6','M072_0.6','M073_0.6','M074_0.6','M075_0.6','M076_0.6','M077_0.6','M078_0.6','M079_0.6','M080_0.6','M081_0.6','M082_0.6','M083_0.6','M084_0.6','M085_0.6','M086_0.6','M087_0.6','M088_0.6','M089_0.6','M090_0.6','M091_0.6','M092_0.6','M093_0.6','M094_0.6','M095_0.6','M096_0.6','M097_0.6','M098_0.6','M099_0.6','M100_0.6','M101_0.6','M102_0.6','M103_0.6','M104_0.6','M105_0.6','M106_0.6','M107_0.6','M108_0.6','M109_0.6','M110_0.6','M111_0.6','M112_0.6','M113_0.6','M114_0.6','M115_0.6','M116_0.6','M117_0.6','M118_0.6','M119_0.6','M120_0.6','M121_0.6','M122_0.6','M123_0.6','M124_0.6','M125_0.6','M126_0.6','M127_0.6','M128_0.6','M129_0.6','M130_0.6','M131_0.6','M132_0.6','M133_0.6','M134_0.6','M135_0.6','M136_0.6','M137_0.6','M138_0.6','M139_0.6','M140_0.6','M141_0.6','M142_0.6','M143_0.6','M144_0.6','M145_0.6','M146_0.6','M147_0.6','M148_0.6','M149_0.6','M150_0.6','M151_0.6','M152_0.6','M153_0.6','M154_0.6','M155_0.6','M156_0.6','M157_0.6','M158_0.6','M159_0.6','M160_0.6','M161_0.6','M162_0.6','M163_0.6','M164_0.6','M165_0.6','M166_0.6','M167_0.6','M168_0.6','M169_0.6','M170_0.6','M171_0.6','M172_0.6','M173_0.6','M174_0.6','M175_0.6','M176_0.6','M177_0.6','M178_0.6','M179_0.6','M180_0.6','M181_0.6','M182_0.6','M183_0.6','M184_0.6','M185_0.6','M186_0.6','M187_0.6','M188_0.6','M189_0.6','M190_0.6','M191_0.6','M192_0.6','M193_0.6','M194_0.6','M195_0.6','M196_0.6','M197_0.6','M198_0.6','M199_0.6','M200_0.6','M201_0.6','M202_0.6','M203_0.6','M204_0.6','M205_0.6','M206_0.6','M207_0.6','M208_0.6','M209_0.6','M210_0.6','M211_0.6','M212_0.6','M213_0.6','M214_0.6','M215_0.6','M216_0.6','M217_0.6','M218_0.6','M219_0.6','M220_0.6','M221_0.6','M222_0.6','M223_0.6','M224_0.6','M225_0.6','M226_0.6','M227_0.6','M228_0.6','M229_0.6','M230_0.6','M231_0.6','M232_0.6','M233_0.6','M234_0.6','M235_0.6','M236_0.6','M237_0.6','M238_0.6','M239_0.6','M240_0.6','M241_0.6','M242_0.6','M243_0.6','M244_0.6']

#ertyAct = vb2.reindex(columns=cols)
#ertyAct.to_csv('Act-CISBP-ZScores_Format3.csv')
#print(Act_motifs)